In [24]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = keras.utils.load_img(image_path, target_size=(img_nrows, img_ncols))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)


def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x


In [25]:
# The gram matrix of an image tensor (feature-wise outer product)


def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


# The "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels**2) * (size**2))


# An auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image


def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))


# The 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


def total_variation_loss(x):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))


In [26]:
# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Set up a model that returns the activation values for every layer in
# VGG19 (as a dict).
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [27]:
# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# The layer to use for the content loss.
content_layer_name = "block5_conv2"


def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add total variation loss
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss


In [28]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [32]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import numpy as np
import tensorflow as tf
from keras.applications import vgg19
from PIL import Image

base_image_path = '/content/drive/MyDrive/2024-2025/Deep_Learning_Assignments/Datasets/custom_images/'
style_ref_path = '/content/style_refs/'
images = {'image1': base_image_path + 'image1_me_and_sis.jpg',
          'image2': base_image_path + 'image2_me_dad_sis.jpg',
          'image3': base_image_path + 'image3_iguazu_falls.jpg',
          'image4': base_image_path + 'image4_purple_flowers.jpg',
          'image5': base_image_path + 'image5_mural.jpg',
          'image6': base_image_path + 'image6_cake.jpg',
          'image7': base_image_path + 'image7_lucky_dog.jpg'}
style_refs = {'image1': style_ref_path + 'scream.jpg',
              'image2': style_ref_path + 'scream.jpg',
              'image3': style_ref_path + 'starry_night.jpg',
              'image4': style_ref_path + 'wave.jpg',
              'image5': style_ref_path + 'starry_night.jpg',
              'image6': style_ref_path + 'colorful.jpg',
              'image7': style_ref_path + 'black_white.jpg'}

for image_name, image_path in images.items():
  for style_ref_name, style_ref_path in style_refs.items():
    if image_name == style_ref_name and ( image_name == 'image7'):
      style_reference_image_path = style_ref_path
      result_prefix = image_name + '_generated'
      total_variation_weight = 1e-6
      style_weight = 1e-6
      content_weight = 2.5e-8
      width, height = keras.utils.load_img(images[image_name]).size
      img_nrows = 400
      img_ncols = int(width * img_nrows / height)
      img_path = images[image_name]
      img = Image.open(img_path)
      style_image = Image.open(style_reference_image_path)
      img_resized = img.resize(style_image.size)
      img_resized.save(img_path, 'JPEG')
      optimizer = keras.optimizers.SGD(
          keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96)
          )
      base_image = preprocess_image(img_path)
      style_reference_image = preprocess_image(style_reference_image_path)
      combination_image = tf.Variable(preprocess_image(img_path))
      iterations = 4000
      for i in range(1, iterations + 1):
        loss, grads = compute_loss_and_grads(
          combination_image, base_image, style_reference_image
        )
        optimizer.apply_gradients([(grads, combination_image)])
        if i % 100 == 0:
          print("Iteration %d: loss=%.2f" % (i, loss))
          img = deprocess_image(combination_image.numpy())
          fname = result_prefix + "_at_iteration_%d.png" % i
          keras.utils.save_img(f'/content/results/{image_name}/results_during_optimization/' + fname, img)








Iteration 100: loss=2058.96
Iteration 200: loss=1247.31
Iteration 300: loss=978.76
Iteration 400: loss=850.17
Iteration 500: loss=775.63
Iteration 600: loss=726.76
Iteration 700: loss=692.00
Iteration 800: loss=665.94
Iteration 900: loss=645.47
Iteration 1000: loss=628.97
Iteration 1100: loss=615.33
Iteration 1200: loss=603.83
Iteration 1300: loss=593.99
Iteration 1400: loss=585.50
Iteration 1500: loss=578.08
Iteration 1600: loss=571.51
Iteration 1700: loss=565.65
Iteration 1800: loss=560.41
Iteration 1900: loss=555.70
Iteration 2000: loss=551.44
Iteration 2100: loss=547.57
Iteration 2200: loss=544.04
Iteration 2300: loss=540.83
Iteration 2400: loss=537.87
Iteration 2500: loss=535.14
Iteration 2600: loss=532.61
Iteration 2700: loss=530.26
Iteration 2800: loss=528.08
Iteration 2900: loss=526.05
Iteration 3000: loss=524.16
Iteration 3100: loss=522.40
Iteration 3200: loss=520.75
Iteration 3300: loss=519.21
Iteration 3400: loss=517.76
Iteration 3500: loss=516.40
Iteration 3600: loss=515.12

In [33]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

# Specifying the directory
results_directory = '/content/results'
style_refs_directory = '/content/style_refs'


# Writing files to a zipfile
with zipfile.ZipFile('results.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(results_directory, zipf)
with zipfile.ZipFile('style_refs.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(style_refs_directory, zipf)